In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import seed
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import cv2
import os

from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Activation

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, GlobalMaxPooling2D
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Nadam, Adam, SGD, Adadelta
from tensorflow.keras.preprocessing import image
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

In [ ]:
#Определение наиболее часто встречающегося разрешения у изображений

images_name_train = []
images_name_val = []

clases = [1, 2, 3]

for i in clases:
    path = '/content/drive/MyDrive/ВКР/train/' + str(i)
    path_val = '/content/drive/MyDrive/ВКР/val/' + str(i)
    images_name_train.append(os.listdir(path))
    images_name_val.append(os.listdir(path_val))

images_train = []
images_val = []

for i in range(len(clases)):
    for j in images_name_val[i]:
        path = '/content/drive/MyDrive/ВКР/val/' + str(clases[i]) + '/' + str(j)
        im = np.array(Image.open(path)) 
        images_val.append(im)

hw_val = []
height_val = []
width_val = []
for i in range(len(images_val)):
    hw_val.append(list(((images_val[i].shape[0], images_val[i].shape[1]))))
    height_val.append(images_val[i].shape[0])
    width_val.append(images_val[i].shape[1])

width_val_pd = pd.Series(width_val)
width_val_pd.value_counts()[:5]

In [ ]:
!pip install keras-rectified-adam
from keras_radam import RAdam

In [ ]:
#Построение первой базовой модели
#Фиксирую сид для последующей воспроизводимости результатов
seed(42)
tf.random.set_seed(42)

num_classes = 3

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (800, 640, 3), padding = "same", kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding = "same", kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32, kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

keras.utils.plot_model(model, show_shapes = True)

Определение генераторов для подгрузки изображений по батчам из папки:
datagen_train = ImageDataGenerator(preprocessing_function = tf.keras.applications.densenet.preprocess_input,
                                   rotation_range = 20,
                                   horizontal_flip = True)

datagen_val = ImageDataGenerator(preprocessing_function = tf.keras.applications.densenet.preprocess_input)

input_shape = (800, 640, 3)
batch_size = 4

train_dir = '/content/drive/MyDrive/ВКР/Красота ремонта/Спальня/train'
train_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size = input_shape[:-1],
    batch_size = batch_size,
    interpolation = "bilinear",
    class_mode = 'categorical',
    seed = 42
    )

val_dir = '/content/drive/MyDrive/ВКР/Красота ремонта/Спальня/val'
val_generator = datagen_val.flow_from_directory(
    val_dir,
    target_size = input_shape[:-1],
    batch_size = batch_size,
    interpolation = "bilinear",
    class_mode = 'categorical',
    seed = 42,
    shuffle = True # False необходимо для отображения имени изображения
    )

#Обучение модели, определение ее параметров, чекпоинтов
epochs = 30

checkpoint = ModelCheckpoint("/content/basic_model_bedroom_1.h5", monitor = 'val_accuracy',
                             verbose = 1, save_best_only = True, mode = 'max') # сохраняет лучшую модель по точности на валидации
callbacks_list = [checkpoint]

opt = RAdam(learning_rate = 1e-4)

model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 697 // batch_size,
    epochs = epochs,
    validation_data = val_generator,
    validation_steps = 241 // batch_size,
    callbacks = callbacks_list
)

#Построение графика точности на тренировочной и валидационной части датасета
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

In [ ]:
# Построение модели с использованием технологии переноса обучения
def scheduler(accuracy, lr):
    if accuracy >= 0.8:
        lr = 3 * 1e-5
        return lr
    if (accuracy >= 0.9 and accuracy < 0.95):
        lr = 1e-5
        return lr
    if (accuracy >= 0.95 and accuracy < 0.98):
        lr = 3 * 1e-6
        return lr
    if(accuracy >= 0.98):
        lr = 1e-6
        return lr
    else:
        return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Transfer learning

use_pretrain_model = False

if (use_pretrain_model == True):
    model = load_model("/content/drive/MyDrive/ВКР/Красота ремонта/Студия/densenet201_studio_1.h5", custom_objects = {'RAdam': RAdam})

else:
    densenet201 = tf.keras.applications.DenseNet201(
        input_shape = input_shape,
        weights = 'imagenet',
        include_top = False
    )

    densenet201.trainable = True

    model = tf.keras.Sequential([
        densenet201,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(3, activation = 'softmax')
    ])

epochs = 1000

checkpoint = ModelCheckpoint("/content/drive/MyDrive/ВКР /Красота ремонта/Студия/densenet201_studio_1.h5", monitor = 'val_accuracy',
                                                         verbose = 1, save_best_only = True, mode = 'max') # сохраняет лучшую модель по точности на валидации
callbacks_list = [checkpoint, lr_scheduler]

opt = RAdam(learning_rate = 1e-4)

model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['accuracy'])
model.fit_generator(
        train_generator,
        steps_per_epoch = 278 // batch_size,
        epochs = 1000,
        validation_data = val_generator,
        validation_steps = 98 // batch_size,
        callbacks = callbacks_list
)

In [ ]:
#Вывод тех изображений, на которых модель ошиблась

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

k = 0
val_labels = []
val_labels_pred = []

for x, y in val_generator:

    pred_batch = model.predict(x)
    for i in range(len(y)):
        label = int(np.argmax(y[i]))
        pred = int(np.argmax(pred_batch[i]))
        val_labels.append(label)
        val_labels_pred.append(pred)
        if(label != pred):
            idx = (val_generator.batch_index - 1) * batch_size
            print("Метка: ", label, "Предсказание:", pred, pred_batch[i])
            print(val_generator.filenames[idx + i])
            plt.figure(figsize=(13, 11))
            plt.imshow(x[i] * std + mean)
            plt.show()
    k = k + 1
    if (k == 61):
        break

In [ ]:
#Вывод точности модели и матрицы ошибок
print(accuracy_score(val_labels, val_labels_pred))
print(confusion_matrix(val_labels, val_labels_pred))

In [ ]:
# Построение ансамбля из двух моделей + XGBoost/LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
k = 0
train_labels = []
train_labels_pred = []
X_train = []

for batch1, batch2 in zip(train_generator, train_generator2):
    x1, y = batch1
    x2, y = batch2
    pred_batch_1 = model1.predict(x1) # предсказания первой модели по батчу из 4 изображений, размер 4 строки x 3 столбца с вероятностями
    pred_batch_2 = model2.predict(x2)

    for i in range(len(y)):
        X_train.append(list(pred_batch_1[i]) + list(pred_batch_2[i])) # объединение двух массивов в один массив признаков
    for i in range(len(y)): # Прохожу по элементам батча
        label = np.argmax(y[i])
        train_labels.append(label)
    k = k + 1
    if (k == 175):
        break

k = 0
val_labels = []
train_labels_pred = []
X_val = []

for batch1, batch2 in zip(val_generator, val_generator2):
    x1, y = batch1
    x2, y = batch2
    pred_batch_1 = model1.predict(x1) # предсказания первой модели по батчу из 4 изображений, размер 4 строки x 3 столбца с вероятностями
    pred_batch_2 = model2.predict(x2)

    for i in range(len(y)):
        X_val.append(list(pred_batch_1[i]) + list(pred_batch_2[i]))
    for i in range(len(y)): # Прохожу по элементам батча
        label = np.argmax(y[i])
        val_labels.append(label)
    k = k + 1
    if (k == 61):
        break

In [ ]:
X_train = np.array(X_train)
xgb_model = XGBClassifier(objective = 'multi:softmax').fit(X_train, train_labels)
predictions = xgb_model.predict(X_val)

print(accuracy_score(val_labels, predictions))

In [ ]:
lr = LogisticRegression(solver='lbfgs',random_state=17)
lr.fit(X_train, train_labels)
predictions = lr.predict(X_val)

print(accuracy_score(val_labels, predictions))

In [ ]:
max_depth = np.arange(1, 11)
subsample = np.linspace(0.1, 0.9, 9)
n_estimators = np.arange(1, 10)
clf = GridSearchCV(XGBClassifier(random_state=22), {'subsample': subsample, 'max_depth': max_depth, 'n_estimators': n_estimators}, cv = 3, scoring='accuracy')
clf.fit(X_train, train_labels)
print("Best param:", clf.best_params_)